In [48]:
import cv2

# Load the pre-trained car classifier
car_cascade = cv2.CascadeClassifier('cars.xml')

# Open the video file
video = cv2.VideoCapture('Test Video.mp4.zip')

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect cars in the frame
    cars = car_cascade.detectMultiScale(gray, 1.1, 1)

    # Draw rectangles around detected cars
    for (x, y, w, h) in cars:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # Display the frame with detections
    cv2.imshow('Car Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release video capture and close windows
video.release()
cv2.destroyAllWindows()



# Speed Calculation

In [52]:
import time

# Define pixel positions of Line A and Line B (adjust according to your video)
line_a_y = 150
line_b_y = 300

# Distance between Line A and B in meters (you must manually estimate this)
distance_m = 10  

# Dictionary to hold entry times of cars (key = car ID or position)
car_times = {}

# Speed limit in km/h
speed_limit = 60

# Loop through detected cars from previous detection frame
for (x, y, w, h) in cars:
    car_center_y = y + h // 2
    car_id = f"{x}_{y}"  # simple way to give a unique-ish ID

    if line_a_y - 5 < car_center_y < line_a_y + 5 and car_id not in car_times:
        # Car crossed Line A, store the time
        car_times[car_id] = time.time()

    elif line_b_y - 5 < car_center_y < line_b_y + 5 and car_id in car_times:
        # Car crossed Line B, calculate speed
        time_diff_sec = time.time() - car_times[car_id]
        speed_kmph = (distance_m / time_diff_sec) * 3.6
        print(f"Car {car_id} speed: {speed_kmph:.2f} km/h")

        if speed_kmph > speed_limit:
            print(f"⚠️ Car {car_id} exceeded speed limit! 🚨")
            # issue_challan()  # your custom function

        # Cleanup the car ID from the dict
        del car_times[car_id]


# License Plate Recognition

In [56]:
!pip install easyocr


   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 2.9/2.9 MB 18.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 16.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   - -------------------------------------- 5.5/204.1 MB 24.0 MB/s eta 0:00:09
   -- ------------------------------------- 10.7/204.1 MB 25.8 MB/s eta 0:00:08
   --- ------------------------------------ 17.0/204.1 MB 27.5 MB/s eta 0:00:07
   ---- ----------------------------------- 23.1/204.1 MB 28.1 MB/s eta 0:00:07
   ----- ---------------------------------- 28.0/204.1 MB 27.4 MB/s eta 0:00:07
   ------ --------------------------------- 33.6/204.1 MB 27.0 MB/s eta 0:00:07
   ------- -------------------------------- 38.8/204.1 MB 27.1 MB/s eta 0:00:07
   -------- ------------------------------- 44.8/204.1 MB 27.2 MB/s eta 0:00:0

In [ ]:
import easyocr

# Initialize OCR reader (only once)
reader = easyocr.Reader(['en'])

# Assuming 'car_roi' contains the cropped car or license plate region from the video
ocr_results = reader.readtext(car_roi)

for detection in ocr_results:
    bbox, text, confidence = detection
    if confidence > 0.3:  # Optional: filter by confidence
        print("Detected License Plate:", text)


# Auto Deduct from Account(Mock)

In [1]:
# Sample account database (plate: balance)
accounts = {"MP12AB1234": 5000}

# Function to deduct fine
def deduct_challan(plate, fine=500):
    if plate in accounts:
        if accounts[plate] >= fine:
            accounts[plate] -= fine
            print(f"✅ ₹{fine} deducted from {plate}. Remaining Balance: ₹{accounts[plate]}")
        else:
            print(f"❌ Insufficient funds for {plate}. Available: ₹{accounts[plate]}, Required: ₹{fine}")
    else:
        print(f"❌ License plate {plate} not found in the account database.")


# Storing Challans in SQLite

In [3]:
import sqlite3
from datetime import datetime

# Connect to SQLite database (or create if it doesn't exist)
conn = sqlite3.connect("challan.db")
cursor = conn.cursor()

# Create table for challan records (if not already present)
cursor.execute("""
CREATE TABLE IF NOT EXISTS challans (
    plate TEXT,
    fine_amount INTEGER,
    date TEXT
)
""")

# Function to deduct fine and store the challan record
def deduct_challan(plate, fine=500):
    # Check if plate exists in the account (replace with actual check for balance)
    if plate in accounts:
        if accounts[plate] >= fine:
            accounts[plate] -= fine
            print(f"✅ ₹{fine} deducted from {plate}. Remaining Balance: ₹{accounts[plate]}")
            
            # Get current time for the challan record
            date_of_deduction = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            # Insert the challan record into the SQLite table
            cursor.execute("""
            INSERT INTO challans (plate, fine_amount, date) 
            VALUES (?, ?, ?)
            """, (plate, fine, date_of_deduction))
            conn.commit()
            
            print(f"✅ Challan record stored: {plate} fined ₹{fine} on {date_of_deduction}")
        else:
            print(f"❌ Insufficient funds for {plate}. Available: ₹{accounts[plate]}, Required: ₹{fine}")
    else:
        print(f"❌ License plate {plate} not found in the account database.")

# Example account database (for simplicity, using in-memory dictionary)
accounts = {"MP12AB1234": 5000}

# Example call
deduct_challan("MP12AB1234", 500)

# Close the SQLite connection when done
conn.close()


✅ ₹500 deducted from MP12AB1234. Remaining Balance: ₹4500
✅ Challan record stored: MP12AB1234 fined ₹500 on 2025-04-08 00:59:56


Conclusion:
This project showcases an AI-powered vehicle monitoring system using Python, OpenCV, and OCR. It detects moving cars in a video, calculates their speed, identifies license plates using EasyOCR, deducts fines for overspeeding, and logs the event in an SQLite database. The project demonstrates key applications in traffic surveillance, automation, and computer vision.

Tools Used: Python, OpenCV, EasyOCR, SQLite